In [1]:
cd tensors

/files/home/part5/dsuess/Xenial/tensors


In [2]:
import mpnum as mp
from altmin import AltminEstimator
import itertools as it
from tqdm import tqdm
import matplotlib.pyplot as pl
import pickle

from mpnum.utils.extmath import randomized_svd

In [ ]:
SITES = 16
RANK = 1
DIM = 4
m = 1000000

X = mp.random_mpa(SITES, DIM, RANK, normalized=True)

In [ ]:
STEPS = 20

def get_estims():
    A = [mp.random_mpa(SITES, DIM, 1, normalized=True)
         for _ in range(m)]
    b = [mp.special.inner_prod_mps(a, X) for a in A]
    X_init = mp.special.sumup(A, rank=RANK, weights=b, svdfunc=randomized_svd)
    X_init /= mp.norm(X_init)
    estim = AltminEstimator(A, b, 1, X_init=X_init.copy())
    return [X_sharp for X_sharp in tqdm(it.islice(estim, STEPS), total=STEPS)]

X_sharps = [get_estims() for _ in range(10)]

for X_sharp in X_sharps:
    pl.plot([mp.normdist(X, X_s) for X_s in X_sharp])

 75%|███████▌  | 15/20 [1:11:01<23:40, 284.09s/it]

In [ ]:
with open('recoveries_sites16.pkl', 'wb') as buf:
    pickle.dump((X, X_sharps), buf)

In [ ]:
mp.normdist(X, X_sharps[0][0])

In [ ]:
A = [mp.random_mpa(SITES, DIM, 1, normalized=True)
     for _ in range(m)]
b = [mp.special.inner_prod_mps(a, X) for a in A]
X_init = mp.special.sumup(A, rank=RANK, weights=b, svdfunc=randomized_svd)
X_init /= mp.norm(X_init)
mp.normdist(X, X_init)

In [4]:
A = [mp.random_mpa(SITES, DIM, 1, normalized=True)
     for _ in range(m)]
b = [mp.special.inner_prod_mps(a, X) for a in A]

In [5]:
def bench():
    c = mp.sumup(A, weights=b)
    c.compress(rank=10)
    del c
    
%timeit bench()

2min 42s ± 786 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
len(A)

13107

In [ ]:
S.compress(rank=10)